<a href="https://colab.research.google.com/github/ashutosh3060/Agentic-RAG/blob/main/Notebooks/RAG/RAG_with_Multi_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* **Purpose of the notebook:** This notebook is meant for multi-query RAG Architecture and working on it
  * **What happens with a Multi-Query Translation ?**    
  Ans: With multi-query translation, an AI system uses a Large Language Model (LLM) to generate multiple versions of a user's query, then retrieves documents for each version, and finally combines the unique documents to provide a richer, more comprehensive context for the LLM to generate a response

## Table of Contents
### 0. Pre-requisites
### 1. Multi-Query RAG Architecture
### 2.
### 3.
### 4.

### 0. Pre-requisites

Option 1: Install all the dependent libraries inside a virtual environment

* Install virtual environment library
* Create a virtual environment
* Activate the environment
* Install the required libraries from requirements.txt file

Option 2: Install dependent libraries onn the global python environment


#### Option 1: Install all the dependent libraries inside a virtual environment

In [11]:
!pip install virtualenv
!virtualenv "rag_venv" # To set up the env

!source /content/rag_venv/bin/activate # To activate the environment and download all its dependencies and packages


created virtual environment CPython3.11.11.final.0-64 in 603ms
  creator CPython3Posix(dest=/content/rag_venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.0.1, setuptools==75.8.0, wheel==0.45.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [12]:
# !source /content/myenv/bin/activate; pip3 list

In [13]:
# ! which python

In [30]:
# Install all packages
! pip install -r /content/requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.3 MB/s eta 0:00:00


#### Option 2: Install dependent libraries onn the global python environment

In [15]:
# ! pip install --quiet pinecone-client python-dotenv langchain langchain-community langchain-core langchain-openai beautifulsoup4 tiktoken numpy


### Environment

(1) Packages

In [18]:
import os
from dotenv import load_dotenv

# Load all environment variables from .env file
load_dotenv()

# Access the environment variables (Langchain)
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_endpoint = os.getenv('LANGCHAIN_ENDPOINT')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

# LLM
openai_api_key = os.getenv('OPENAI_API_KEY')

# Pinecone Vector Database
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_host = os.getenv('PINECONE_API_HOST')
index_name = os.getenv('PINECONE_INDEX_NAME')

In [19]:
os.environ['LANGCHAIN_TRACING_V2'] = langchain_tracing_v2
os.environ['LANGCHAIN_ENDPOINT'] = langchain_endpoint
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

In [49]:
os.environ['OPENAI_API_KEY'] = openai_api_key
openai_model = "gpt-3.5-turbo"

#Pinecone keys
os.environ['PINECONE_API_KEY'] = pinecone_api_key
os.environ['PINECONE_API_HOST'] = pinecone_api_host
os.environ['PINECONE_INDEX_NAME'] = index_name

In [50]:
index_name

'ragtest'

In [22]:
print(langchain_tracing_v2)
print(langchain_endpoint)
print(langchain_api_key)

true
https://api.smith.langchain.com
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


Pinecone Init

In [26]:
import time
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
index_name = "ragtest"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
  pc.create_index(
      name=index_name,
      dimension=1536,
      metric="cosine",
      index_type="hnsw",
      sepc=ServerlessSpec(cloud="aws", region="us-east-1"),
  )
  while not pc.describe_index(index_name).status["ready"]:
    time.sleep(1)

index = pc.Index(index_name)

## Understand in Detail

### 1. Index

In [46]:
# Load blog
import bs4
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from pprint import pprint

#### INDEXING ####

# Load Document (Uploading one file at a time)
pdf_file_path = "/content/langchain_turing.pdf"
loader = PyPDFLoader(pdf_file_path)

docs = loader.load()

# Upload muiltiple PDF files from a directory
# pdf_file_paths = <enter your path here>
# loader = PyPDFDirectoryLoader(pdf_file_paths)

# docs_dir = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000,
    chunk_overlap=500)

# Make splits
splits = text_splitter.split_documents(docs)

# Index
vectorstore = PineconeVectorStore.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
    index_name=index_name
)

retriever = vectorstore.as_retriever()

* Prompt

In [51]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(model_name=openai_model, temperature=0.1)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [52]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "How does LangChain leverage modular components like LangGraph, LangSmith, and LangServe to address challenges in building scalable and secure LLM-powered applications?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

7

In [55]:
# docs

In [33]:
# Load Documents

# loader = WebBaseLoader(
#     web_paths = ("https://greenly.earth/en-gb/blog/ecology-news/climate-change-in-2022-where-do-we-stand",),
#     bs_kwargs = dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#             )
#         )
# )

loader = WebBaseLoader("https://greenly.earth/en-gb/blog/ecology-news/climate-change-in-2022-where-do-we-stand")

docs = loader.load()

In [34]:
# Split

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [35]:
# Embed

vectorstore = PineconeVectorStore.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
    index_name=index_name
)

retriever = vectorstore.as_retriever()

* Retrieval & Generation

In [36]:
# Prompt

prompt = hub.pull("rlm/rag-prompt")
# prompt = hub.pull("rlm/rag-prompt:50442af1") --Another prompt (check if it is the same, check quickly with results difference)

In [37]:
# LLM

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1) #Temperature is usually a value between 0 and 1, where 0 is completely deterministic and 1 is highly random

In [38]:
# Post Processing
def format_docs(docs):
  '''
  This function formats the retrieved output from the vectorstore as below.
  Get the doc content and format it in such a way that there are 2 new lines between every doc content
  '''
  return "\n\n".join(doc.page_content for doc in docs)

In [39]:
# Chain

# In the below chain, first input is passed through RunnablePassthrough, then the retriever retrieves the information from vector store
# The output from the vector store is post processed
# The processed output from vector store is then fed to the prompt for prompt engineering or specific prompt styling
# The LLM receives the prompt in the specified style and generates the text / answer.
# The LLM's output is then formatted through Langchain's String Output parser and being returned.

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [40]:
# # Question
# pprint(rag_chain.invoke("How does LangChain use vector stores for efficient data retrieval?"))

In [41]:
# Question
pprint(rag_chain.invoke("worst projection for climate change in 2024"))

('The worst projection for climate change in 2024 indicates that there is '
 'almost a 50% chance for the average global temperature to rise above 1.5°C '
 'during the five-year period from 2022 to 2026. Additionally, 2024 is '
 'expected to be exceptionally warm, following record-breaking temperatures in '
 '2023. This trend highlights the ongoing worsening of climate change due to '
 'high greenhouse gas emissions.')


* End of Notebook